In [247]:
import pandas as pd
import numpy as np
import random

In [248]:
# Cargar los datasets limpios subidos recientemente
gym = pd.read_csv("C:/Users/josen/Documents/MASTER/TFM/Datasets/gym.csv")
members = pd.read_csv("C:/Users/josen/Documents/MASTER/TFM/Datasets/members.csv")

In [249]:
gym.head()

,Title,Type,BodyPart,Equipment,Level,BodyGroup
0,Partner plank band row,Strength,Abdominals,Bands,Intermediate,Core
1,Banded crunch isometric hold,Strength,Abdominals,Bands,Intermediate,Core
2,FYR Banded Plank Jack,Strength,Abdominals,Bands,Intermediate,Core
3,Banded crunch,Strength,Abdominals,Bands,Intermediate,Core
4,Crunch,Strength,Abdominals,Bands,Intermediate,Core


In [250]:
members['Level'] = members['Level'].replace('Expert', 'Advanced')
gym['Level'] = gym['Level'].replace('Expert', 'Advanced')

In [251]:
gym['Level'].unique()

array(['Intermediate', 'Beginner', 'Advanced'], dtype=object)

In [252]:
members['Level'].unique()

array(['Advanced', 'Intermediate', 'Beginner'], dtype=object)

In [253]:
gym['BodyPart'].unique()

array(['Abdominals', 'Adductors', 'Abductors', 'Biceps', 'Calves',
       'Chest', 'Forearms', 'Glutes', 'Hamstrings', 'Lats', 'Lower Back',
       'Middle Back', 'Traps', 'Neck', 'Quadriceps', 'Shoulders',
       'Triceps'], dtype=object)

In [254]:
gym["BodyGroup"].unique()

array(['Core', 'Legs', 'Upper Body', 'Back', 'Other'], dtype=object)

In [255]:
gym['Type'].unique()

array(['Strength', 'Plyometrics', 'Cardio', 'Stretching', 'Powerlifting',
       'Strongman', 'Olympic Weightlifting'], dtype=object)

In [256]:
gym.groupby(['BodyPart', 'BodyGroup', 'Type'])['Title'].apply(list).reset_index()
print(gym)

                                               Title      Type    BodyPart  \
0                             Partner plank band row  Strength  Abdominals   
1                       Banded crunch isometric hold  Strength  Abdominals   
2                              FYR Banded Plank Jack  Strength  Abdominals   
3                                      Banded crunch  Strength  Abdominals   
4                                             Crunch  Strength  Abdominals   
...                                              ...       ...         ...   
2913                            EZ-bar skullcrusher-  Strength     Triceps   
2914  Lying Close-Grip Barbell Triceps Press To Chin  Strength     Triceps   
2915          EZ-Bar Skullcrusher - Gethin Variation  Strength     Triceps   
2916                                TBS Skullcrusher  Strength     Triceps   
2917                     30 Arms EZ-Bar Skullcrusher  Strength     Triceps   

         Equipment         Level   BodyGroup  
0            Ban

In [257]:
members.head()

,Age,Gender,Weight_kg,Height_m,Workout_time,Calories_Burned,Type,Fat_Percentage,Frequency,Level,BMI
0,56,Male,88.3,1.71,1.69,1313.0,Yoga,12.6,4,Advanced,30.20
1,46,Female,74.9,1.53,1.30,883.0,HIIT,33.9,4,Intermediate,32.00
2,32,Female,68.1,1.66,1.11,677.0,Cardio,33.4,4,Intermediate,24.71
3,25,Male,53.2,1.70,0.59,532.0,Strength,28.8,3,Beginner,18.41
4,38,Male,46.1,1.79,0.64,556.0,Strength,29.2,3,Beginner,14.39


In [258]:
members['Type'].unique()

array(['Yoga', 'HIIT', 'Cardio', 'Strength'], dtype=object)

In [259]:
members.groupby(['Type', 'Level'])['Age'].apply(list).reset_index()
print(members)

     Age  Gender  Weight_kg  Height_m  Workout_time  Calories_Burned  \
0     56    Male       88.3      1.71          1.69           1313.0   
1     46  Female       74.9      1.53          1.30            883.0   
2     32  Female       68.1      1.66          1.11            677.0   
3     25    Male       53.2      1.70          0.59            532.0   
4     38    Male       46.1      1.79          0.64            556.0   
..   ...     ...        ...       ...           ...              ...   
968   24    Male       87.1      1.74          1.57           1364.0   
969   25    Male       66.6      1.61          1.38           1260.0   
970   59  Female       60.4      1.76          1.72            929.0   
971   32    Male      126.4      1.83          1.10            883.0   
972   46    Male       88.7      1.63          0.75            542.0   

         Type  Fat_Percentage  Frequency         Level    BMI  
0        Yoga            12.6          4      Advanced  30.20  
1      

In [260]:
type_map = {
    'Yoga': 'Stretching',
    'HIIT': 'Plyometrics',
    'Cardio': 'Cardio',
    'Strength': 'Strength'
}

members['Type_Mapped'] = members['Type'].map(type_map)
gym['Type_Mapped'] = gym['Type'].map(type_map).fillna(gym['Type'])

In [261]:
# Añadir IDs para referencia
members['user_id'] = members.index
gym['exercise_id'] = gym.index

In [262]:
# Subconjuntos para emparejamiento
members_expanded = members[['user_id', 'Type', 'Level']]
gym_expanded = gym[['exercise_id', 'Type', 'Level']]

In [263]:
# Merge filtrado por coincidencia en Type y Level
entrenamiento = members_expanded.merge(
    gym_expanded,
    on=['Type', 'Level'],
    how='inner'
)

In [264]:
# Añadir características completas de usuario y ejercicio
entrenamiento = entrenamiento.merge(members, on='user_id', how='left')
entrenamiento = entrenamiento.merge(gym, on='exercise_id', how='left')

# Mostrar tamaño y ejemplo
entrenamiento.shape, entrenamiento.head(3)

((292842, 23),
    user_id  Type_x       Level_x  exercise_id  Age  Gender  Weight_kg  \
 0        2  Cardio  Intermediate          149   32  Female       68.1   
 1        2  Cardio  Intermediate          349   32  Female       68.1   
 2        2  Cardio  Intermediate          884   32  Female       68.1   
 
    Height_m  Workout_time  Calories_Burned  ...       Level_y    BMI  \
 0      1.66          1.11            677.0  ...  Intermediate  24.71   
 1      1.66          1.11            677.0  ...  Intermediate  24.71   
 2      1.66          1.11            677.0  ...  Intermediate  24.71   
 
    Type_Mapped_x                  Title    Type    BodyPart   Equipment  \
 0         Cardio            Roc Everest  Cardio  Abdominals       Other   
 1         Cardio  HM Running Man Crunch  Cardio  Abdominals  Bodyweight   
 2         Cardio         Agility Ladder  Cardio      Calves       Other   
 
           Level BodyGroup Type_Mapped_y  
 0  Intermediate      Core        Cardio  
 

In [265]:
entrenamiento.head()

,user_id,Type_x,Level_x,exercise_id,Age,Gender,Weight_kg,Height_m,Workout_time,Calories_Burned,...,Level_y,BMI,Type_Mapped_x,Title,Type,BodyPart,Equipment,Level,BodyGroup,Type_Mapped_y
0,2,Cardio,Intermediate,149,32,Female,68.1,1.66,1.11,677.0,...,Intermediate,24.71,Cardio,Roc Everest,Cardio,Abdominals,Other,Intermediate,Core,Cardio
1,2,Cardio,Intermediate,349,32,Female,68.1,1.66,1.11,677.0,...,Intermediate,24.71,Cardio,HM Running Man Crunch,Cardio,Abdominals,Bodyweight,Intermediate,Core,Cardio
2,2,Cardio,Intermediate,884,32,Female,68.1,1.66,1.11,677.0,...,Intermediate,24.71,Cardio,Agility Ladder,Cardio,Calves,Other,Intermediate,Legs,Cardio
3,2,Cardio,Intermediate,993,32,Female,68.1,1.66,1.11,677.0,...,Intermediate,24.71,Cardio,Swimming,Cardio,Chest,Bodyweight,Intermediate,Upper Body,Cardio
4,2,Cardio,Intermediate,1729,32,Female,68.1,1.66,1.11,677.0,...,Intermediate,24.71,Cardio,Rower,Cardio,Middle Back,Machine,Intermediate,Back,Cardio


In [266]:
print(entrenamiento.columns)

Index(['user_id', 'Type_x', 'Level_x', 'exercise_id', 'Age', 'Gender',
       'Weight_kg', 'Height_m', 'Workout_time', 'Calories_Burned', 'Type_y',
       'Fat_Percentage', 'Frequency', 'Level_y', 'BMI', 'Type_Mapped_x',
       'Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'BodyGroup',
       'Type_Mapped_y'],
      dtype='object')


In [267]:
columnas_a_mostrar = [col for col in entrenamiento.columns if col.startswith('Type') or col.startswith('Level')]
print(columnas_a_mostrar)

['Type_x', 'Level_x', 'Type_y', 'Level_y', 'Type_Mapped_x', 'Type', 'Level', 'Type_Mapped_y']


In [268]:
columnas_a_mantener = ['user_id', 'exercise_id', 'Age', 'Gender',
       'Weight_kg', 'Height_m', 'Workout_time', 'Calories_Burned',
       'Fat_Percentage', 'Frequency',  'BMI', 'Type_Mapped_x',
       'Title', 'Type', 'BodyPart', 'Equipment', 'Level', 'BodyGroup',
       ]

entrenamiento = entrenamiento[columnas_a_mantener]
entrenamiento.rename(columns={'Type_Mapped_x': 'Type_Mapped'}, inplace=True)
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292842 entries, 0 to 292841
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_id          292842 non-null  int64  
 1   exercise_id      292842 non-null  int64  
 2   Age              292842 non-null  int64  
 3   Gender           292842 non-null  object 
 4   Weight_kg        292842 non-null  float64
 5   Height_m         292842 non-null  float64
 6   Workout_time     292842 non-null  float64
 7   Calories_Burned  292842 non-null  float64
 8   Fat_Percentage   292842 non-null  float64
 9   Frequency        292842 non-null  int64  
 10  BMI              292842 non-null  float64
 11  Type_Mapped      292842 non-null  object 
 12  Title            292842 non-null  object 
 13  Type             292842 non-null  object 
 14  BodyPart         292842 non-null  object 
 15  Equipment        292842 non-null  object 
 16  Level            292842 non-null  obje

In [269]:
# Guardar el dataset combinado
entrenamiento.to_csv("C:/Users/josen/Documents/MASTER/TFM/SmartLifter.csv", index=False)